<a href="https://colab.research.google.com/github/lawesworks/vision-model-workbench/blob/main/vision_yolo_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# CONFIGURATION
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# interesting note..  with TP4 (GPU), training time is about 12min for 30 epochs..  with CPU only, training is nearly 10 hours.

# User Settings
SAMPLE_IMAGE = "fire.jpg"

# Roboflow Model Settings
Roboflow_Workspace_Name = "-jwzpw"
Roboflow_Project_Name = "continuous_fire"
Roboflow_Project_Version = 1

# YOLO Model Settings
YOLO_Model_Version = "yolov8"
YOLO_Model_Size = "n"

# Training Hyper-parameters
Config_Epochs = 30
Config_Image_Size = 640
Config_Batch_Size = 16

#-------------------------------------------------------------------------------

# Auto-Derived Parameters
Roboflow_Project_Folder = Roboflow_Project_Name+"-"+str(Roboflow_Project_Version)
LATEST_PREDICT_DIR = "runs/detect/predict"
LATEST_TRAIN_DIR   = "runs/detect/train"


print(f"""
===== Training Configuration =====

Workspace        : {Roboflow_Workspace_Name}
Project          : {Roboflow_Project_Name}
Project Folder   : {Roboflow_Project_Folder}
Dataset Version  : {Roboflow_Project_Version}

Model            : {YOLO_Model_Version}
Model Size       : {YOLO_Model_Size}
Epochs           : {Config_Epochs}
Image Size       : {Config_Image_Size}
Batch Size       : {Config_Batch_Size}

==================================
""")

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Load Roboflow API Key
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

from google.colab import userdata
api_key = userdata.get('ROBOFLOW_API_KEY')

if api_key is None:
    raise ValueError("ROBOFLOW_API_KEY not found. Check Colab Secrets.")

print("Roboflow API Key Found")

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Load Libraries
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

from IPython.display import Image, display
import glob
import os

print(f"""
===== Imported Libraries =====

Ipython.display  : Image
Ipython.display  : display
glob
os

==================================
""")

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Install Ultralytics
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

print("Installing Ultralytics (Please wait)\n")

!pip install -q roboflow ultralytics

print("\nCompleted Ultralytics Install")

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# HELPER FUNCTIONS
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

import math
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import Image, display
import torch




# ----------------------------------------------------------------------------------------------------
# Image Grid (Left → Right, Row Wrap) that will create a grid of Images saved to the Predicted Folder
# ----------------------------------------------------------------------------------------------------
import torch

def get_gpu_cv_summary(device_index: int = 0) -> dict:
    """
    Return a concise GPU capability summary for CV / YOLO-style workloads.

    Returns a dict with:
      - property
      - value
      - why_it_matters
    """
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is not available. Enable a GPU runtime or run on a CUDA-capable machine.")

    props = torch.cuda.get_device_properties(device_index)

    # Some attributes may not exist on all builds/versions; use getattr with fallback.
    summary = [
        {
            "property": "name",
            "value": props.name,
            "why_it_matters": "GPU generation & capabilities",
        },
        {
            "property": "total_memory",
            "value": f"{props.total_memory / (1024**3):.2f} GB",
            "why_it_matters": "Max batch size & image resolution",
        },
        {
            "property": "multi_processor_count",
            "value": props.multi_processor_count,
            "why_it_matters": "Parallel throughput",
        },
        {
            "property": "clock_rate",
            "value": f"{getattr(props, 'clock_rate', None) / 1000:.0f} MHz"
                     if getattr(props, "clock_rate", None) is not None else "N/A",
            "why_it_matters": "Kernel execution speed",
        },
        {
            "property": "memory_bus_width",
            "value": getattr(props, "memory_bus_width", "N/A"),
            "why_it_matters": "Data movement speed",
        },
        {
            "property": "warp_size",
            "value": props.warp_size,
            "why_it_matters": "Kernel efficiency",
        },
        {
            "property": "(major.minor)",
            "value": f"{props.major}.{props.minor}",
            "why_it_matters": "CUDA feature support / compute_capability ",
        },
    ]

    return {"device_index": device_index, "gpu_summary": summary}


def print_gpu_cv_summary(device_index: int = 0) -> None:
    """
    Pretty-print the GPU summary in a readable table-like format.
    """
    result = get_gpu_cv_summary(device_index)
    rows = result["gpu_summary"]

    print(f"\nGPU CV/YOLO Capability Summary (device {result['device_index']})")
    print("-" * 78)
    print(f"{'Property':<28} {'Value':<20} {'Why it matters'}")
    print("-" * 78)
    for r in rows:
        print(f"{r['property']:<28} {str(r['value']):<20} {r['why_it_matters']}")
    print("-" * 78)


# ----------------------------------------------------------------------------------------------------
# Image Grid (Left → Right, Row Wrap) that will create a grid of Images saved to the Predicted Folder
# ----------------------------------------------------------------------------------------------------
def show_image_grid_paged(image_paths, cols=5, per_page=20, page=1, figsize_per_cell=3):
    """
    Display images in a true grid, paged.
    - cols: images per row
    - per_page: total images per page
    - page: 1-based page index
    - figsize_per_cell: size multiplier per grid cell
    """
    if not image_paths:
        print("No images to display.")
        return

    start = (page - 1) * per_page
    end = min(start + per_page, len(image_paths))
    page_paths = image_paths[start:end]

    rows = math.ceil(len(page_paths) / cols)
    fig_w = cols * figsize_per_cell
    fig_h = rows * figsize_per_cell

    fig, axes = plt.subplots(rows, cols, figsize=(fig_w, fig_h))
    axes = axes.flatten() if isinstance(axes, (list, tuple)) is False else axes

    # If only one subplot, axes may not be iterable the same way
    try:
        axes = axes.flatten()
    except Exception:
        axes = [axes]

    for ax in axes:
        ax.axis("off")

    for ax, img_path in zip(axes, page_paths):
        img = Image.open(img_path).convert("RGB")
        ax.imshow(img)
        ax.axis("off")

    plt.tight_layout()
    plt.show()

    print(f"Showing images {start+1}–{end} of {len(image_paths)} (page {page})")


# ----------------------------------------------------------------------------------------------------
# Filenames are fine, but only if you keep them small and short - this version truncates titles
# ----------------------------------------------------------------------------------------------------
def show_image_grid_with_short_titles(image_paths, cols=5, per_page=20, page=1, figsize_per_cell=3):
    #import math
    #import matplotlib.pyplot as plt
    #from PIL import Image

    start = (page - 1) * per_page
    end = min(start + per_page, len(image_paths))
    page_paths = image_paths[start:end]

    rows = math.ceil(len(page_paths) / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols*figsize_per_cell, rows*figsize_per_cell))
    try:
        axes = axes.flatten()
    except Exception:
        axes = [axes]

    for ax in axes:
        ax.axis("off")

    for ax, img_path in zip(axes, page_paths):
        img = Image.open(img_path).convert("RGB")
        ax.imshow(img)
        ax.axis("off")
        ax.set_title(os.path.basename(img_path)[:18], fontsize=7)  # truncate title

    plt.tight_layout()
    plt.show()
    print(f"Showing images {start+1}–{end} of {len(image_paths)} (page {page})")


# ----------------------------------------------------------------------------------------------------
# function to determine estimated time of completion
# ----------------------------------------------------------------------------------------------------
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

def estimate_completion_time(minutes_from_now):
    now = datetime.now(ZoneInfo("America/New_York"))
    eta = now + timedelta(minutes=minutes_from_now)

    return {
        "start_time": now.strftime('%Y-%m-%d %H:%M:%S'),
        "estimated_completion": eta.strftime('%Y-%m-%d %H:%M:%S'),
        "minutes_from_now": minutes_from_now
    }




# ----------------------------------------------------------------------------------------------------
# function to estimate training time required for the model
# ----------------------------------------------------------------------------------------------------
def estimate_training_time(
    num_epochs,
    batch_size,
    train_image_count,
    val_image_count=0,
    seconds_per_batch=None,
    seconds_per_epoch=None,
    val_fraction_of_train_time=0.3
):
    """
    Estimate total training time.

    Provide either seconds_per_batch OR seconds_per_epoch (preferrably time for a full epoch).
    If both provided, seconds_per_epoch takes precedence.

    Returns dict with per-epoch and total estimates.
    """
    if seconds_per_epoch is None and seconds_per_batch is None:
        raise ValueError("Provide either seconds_per_batch or seconds_per_epoch.")

    train_batches_per_epoch = math.ceil(train_image_count / batch_size)

    # If user supplied epoch time, convert to per-batch
    if seconds_per_epoch is not None:
        seconds_per_batch = seconds_per_epoch / train_batches_per_epoch

    train_time_per_epoch = train_batches_per_epoch * seconds_per_batch
    val_time_per_epoch = 0
    if val_image_count > 0:
        # Option: scale validation time by image counts, or use fraction
        # Here we estimate validation time proportionally to image counts:
        val_batches_per_epoch = math.ceil(val_image_count / batch_size)
        val_time_per_epoch = val_batches_per_epoch * seconds_per_batch * val_fraction_of_train_time

    total_time_seconds = num_epochs * (train_time_per_epoch + val_time_per_epoch)

    return {
        "train_batches_per_epoch": train_batches_per_epoch,
        "train_time_per_epoch_sec": round(train_time_per_epoch, 2),
        "val_time_per_epoch_sec": round(val_time_per_epoch, 2),
        "total_time_sec": round(total_time_seconds, 2),
        "total_time_min": round(total_time_seconds / 60, 2),
        "total_time_hr": round(total_time_seconds / 3600, 2),
        "seconds_per_batch": round(seconds_per_batch, 4),
    }



# ----------------------------------------------------------------------------------------------------
# function to count images in given folder
# ----------------------------------------------------------------------------------------------------
def count_images(folder):
    extensions = ("*.jpg", "*.jpeg", "*.png")
    count = 0
    for ext in extensions:
        count += len(glob.glob(os.path.join(folder, ext)))
    return count


# ----------------------------------------------------------------------------------------------------
# get latest training path DIR
# ----------------------------------------------------------------------------------------------------
def get_latest_training_path():
    training_dirs = sorted(
        glob.glob("/content/runs/detect/train*"),
        key=os.path.getmtime
    )

    if not training_dirs:
        raise FileNotFoundError("No YOLO training runs found in runs/detect/")

    train_dir = training_dirs[-1]
    print(f"\nUsing training run from: {train_dir}")
    return train_dir



# ----------------------------------------------------------------------------------------------------
# get latest prediction path DIR
# ----------------------------------------------------------------------------------------------------
def get_latest_prediction_path():
  predict_dirs = sorted(
    glob.glob("runs/detect/predict*"),
    key=os.path.getmtime
  )

  PREDICT_DIR = predict_dirs[-1]
  print(f"\n\nUsing predictions from: {PREDICT_DIR}")
  return PREDICT_DIR


# ----------------------------------------------------------------------------------------------------
# get latest model best.pt path
# ----------------------------------------------------------------------------------------------------
def get_latest_pt_path(latest_train_path):
  pt_files = glob.glob(f"{latest_train_path}/weights/best.pt", recursive=True)

  if not pt_files:
      raise FileNotFoundError("No best.pt file found")

  return pt_files[0]

# ----------------------------------------------------------------------------------------------------
# get list of predicted / inferenced images
# ----------------------------------------------------------------------------------------------------
def get_inferenced_images(predict_dir):
  image_paths = glob.glob(os.path.join(predict_dir, "*.jpg")) + \
              glob.glob(os.path.join(predict_dir, "*.png"))

  return sorted(image_paths)


print(f"""
===== Loaded Helper Functions =====

show_image_grid_paged              :
show_image_grid_with_short_titles  :
estimate_completion_time           :
estimate_training_time             :
count_images                       :
get_latest_training_path           :
get_latest_prediction_path         :
get_latest_pt_path                 :
get_inferenced_images              :

==================================
""")

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Load Libraries
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

from IPython.display import Image, display
import glob
import os

print(f"""
===== Imported Libraries =====

Ipython.display  : Image
Ipython.display  : display
glob
os

==================================
""")

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Import the Dataset that will be used to train your model
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

print(f"Importing Dataset: [{Roboflow_Project_Name}, Version: [{Roboflow_Project_Version}] from: [{Roboflow_Workspace_Name}] (Please wait)\n")

from roboflow import Roboflow


rf = Roboflow(api_key=api_key )
project = rf.workspace(Roboflow_Workspace_Name).project(Roboflow_Project_Name)
dataset = project.version(Roboflow_Project_Version).download(YOLO_Model_Version)  # adjust version if needed

print("\nCompleted Dataset Import")

Data_Train_Count = count_images(os.path.join(Roboflow_Project_Folder, "train", "images"))
Data_Valid_Count = count_images(os.path.join(Roboflow_Project_Folder, "valid", "images"))

print(f"""
===== Training Data Statistics =====

Training Image Count               : {Data_Train_Count}
Validation Image Count             : {Data_Valid_Count}

====================================
""")

In [ ]:

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# get location of the yaml file for this project
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

yaml_files = glob.glob(f"/content/{Roboflow_Project_Folder}/data.yaml", recursive=True)

if not yaml_files:
    raise FileNotFoundError("No data.yaml file found")

DATA_YAML_PATH = yaml_files[0]

print(f"Using dataset config: {DATA_YAML_PATH}")

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Estimate Time Required to Complete Training
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

estimate = estimate_training_time(
    num_epochs=Config_Epochs,
    batch_size=Config_Batch_Size,
    train_image_count = Data_Train_Count,
    val_image_count = Data_Valid_Count,
    seconds_per_epoch=25,   # measured full epoch time (NOT per batch)
    val_fraction_of_train_time=0.3
)


# print time estimates
for k, v in estimate.items():
    print(f"{k:<30}: {v}")

# print estimate time of completion
eta_info = estimate_completion_time(estimate["total_time_min"])
for k, v in eta_info.items():
  print(f"{k:<30}: {v}")

print("\n")
print_gpu_cv_summary(0)


In [ ]:
#!nvidia-smi -q
#!nvidia-smi

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# train  model on the data
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


print("Training The Model (Please wait)\n")
print(f"""
===== Training Hyper-parameters =====

Model            : {YOLO_Model_Version}
Model Size       : {YOLO_Model_Size}
Epochs           : {Config_Epochs}
Image Size       : {Config_Image_Size}
Batch Size       : {Config_Batch_Size}
Train Count      : {Data_Train_Count}
Valid Count      : {Data_Valid_Count}

=====================================
""")


from ultralytics import YOLO

DATA_YAML_PATH = DATA_YAML_PATH

model = YOLO("yolov8n.pt")  # small + fast starter model
results = model.train(
    data=DATA_YAML_PATH,
    epochs=Config_Epochs,
    imgsz=Config_Image_Size,
    batch=Config_Batch_Size
)

# get latest training folder - save it to variabble
LATEST_TRAIN_DIR = get_latest_training_path()


In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# get location of the weights for the latest training
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


MODEL_PT_PATH = get_latest_pt_path(LATEST_TRAIN_DIR)

print(f"Using PT File: {MODEL_PT_PATH}")

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# run the model against the test data in your project
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

best_model_path = glob.glob(MODEL_PT_PATH)[-1]
model = YOLO(best_model_path)

model.predict(source = Roboflow_Project_Folder + "/test/images", save=True, conf=0.25)

LATEST_PREDICT_DIR = get_latest_prediction_path()

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Get list of latest predicted / inferenced images
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


image_paths = get_inferenced_images(LATEST_PREDICT_DIR)

print(f"Found {len(image_paths)} predicted images")

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Show images in the Recent Predictions Folder
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# show first 20 images, 5 per row
show_image_grid_paged(image_paths, cols=8, per_page=24, page=1)


In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Show first 20 images with titles, 5 per row
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

show_image_grid_with_short_titles(image_paths, cols=8, per_page=24, page=1)

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# View a screenshot of the training plots (Loss Function plots / Accuracy)
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Image(filename=f'/{LATEST_TRAIN_DIR}/results.png', width=600)

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# View a screenshot of the Confusio Matrix
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Image(filename=f'/{LATEST_TRAIN_DIR}/confusion_matrix.png', width=600)

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# View training metrics
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# model.val(data=DATA_YAML_PATH) # dumps everything (verbose)

metrics = model.val(data=DATA_YAML_PATH)

print("mAP@0.5      :", metrics.box.map50)
print("mAP@0.5:0.95 :", metrics.box.map)
print("Precision    :", metrics.box.mp)
print("Recall       :", metrics.box.mr)


## 📊 Model Evaluation Summary

### Overall Assessment
The model performed **exceptionally well** on the evaluation dataset. All key detection metrics are high, indicating strong object recognition, reliable predictions, and good localization accuracy.

---

### Key Metrics

- **mAP@0.5:** `0.992`
- **mAP@0.5:0.95:** `0.821`
- **Precision:** `0.986`
- **Recall:** `0.964`

---

### Metric Interpretation

#### **mAP@0.5 — Excellent Detection Performance**
- Nearly perfect performance at the 0.5 IoU threshold.
- The model consistently finds objects and places bounding boxes with sufficient overlap.
- Indicates the model has learned the core visual patterns in the dataset extremely well.

#### **mAP@0.5:0.95 — Strong Localization Accuracy**
- Evaluates performance across stricter IoU thresholds.
- The expected drop from mAP@0.5 reflects normal tightening of box placement requirements.
- Suggests good—but not pixel-perfect—bounding box precision, which is typical and acceptable.

#### **Precision — Very High Reliability**
- When the model predicts an object, it is almost always correct.
- Very few false positives.
- Indicates conservative and trustworthy predictions.

#### **Recall — Strong Object Coverage**
- The model detects the vast majority of ground-truth objects.
- Slightly lower than precision, meaning some difficult cases (small, occluded, low-contrast) may be missed.
- Reflects a bias toward avoiding false positives over finding every object.

---

### Precision vs Recall Balance
- **Precision > Recall**  
  The model prioritizes correctness over completeness.
- This is often desirable in scenarios where false positives are more costly than missed detections.

---

### What These Results Suggest

- The dataset is likely **clean and consistently labeled**.
- Transfer learning was effective.
- No obvious signs of underfitting.
- The model is well-tuned for the evaluated data distribution.

---

### Important Caveats

High metrics alone do not guarantee real-world performance. Consider:
- Dataset size and diversity (lighting, angles, environments)
- Potential validation data leakage
- Performance on truly unseen data

---

### Final Takeaway
> **The model demonstrates outstanding performance on the evaluation dataset, with high confidence, strong detection capability, and reliable localization.  
> The next step is to validate robustness using new, unseen data to confirm generalization.
> **The model is biased toward “don’t be wrong,” rather than “find everything.”

---
### Metric Summary

| Metric        | Value | Assessment                     |
|---------------|-------|--------------------------------|
| mAP@0.5       | 0.992 | Outstanding                    |
| mAP@0.5:0.95  | 0.821 | Very strong                    |
| Precision     | 0.986 | Extremely reliable             |
| Recall        | 0.964 | Strong, slightly conservative  |


In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Run inference on an arbitrary image and save it to the sandbox folder
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# I've provided a URL to an image, but you can just as easily upload a file into Colab contents folder
# and use that as your source

#model.predict(source=f"/content/{SAMPLE_IMAGE}", save=True, conf=0.25)

model.predict(
    source="https://github.com/lawesworks/vision-model-workbench/blob/main/images/home%20fire%20hero.jpg?raw=true",
    save=True,
    conf=0.25,
    name=f"sandbox",
    exist_ok=True
)


In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# View a screenshot of the predicted annotated result
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# find the saved image (YOLO keeps original name or auto-generated one)
predicted_images = glob.glob(os.path.join("/content/runs/detect/sandbox", "*.jpg"))

# Sort by last modified time (oldest → newest)
predicted_images.sort(key=os.path.getmtime)
latest_image = predicted_images[-1]

print("Opening:", latest_image,"\n")

Image(
    filename=latest_image,
    width=600
)